In [ ]:
import os
import sqlite3
from shutil import copyfile

db_prefix = 'season_6_'
db_result_file = 'season_6.db' 

db_files = []
for name in os.listdir():
    if name.startswith(db_prefix):
        db_files.append(name)

if not db_files:
    print("No database files were found")
    exit(0)

ordered_files = sorted(db_files, key=lambda name: int(name[len(db_prefix):len(name)-len(os.path.splitext(name)[1])]))
first_db = ordered_files.pop(0)
copyfile(first_db, db_result_file)

comb_db = sqlite3.connect(db_result_file)
for db_name in ordered_files:
    print("Working on DB %s" % db_name)
    comb_db.execute("ATTACH '%s' as dba" % db_name)

    comb_db.execute("BEGIN")
    # SEASON INFO TABLE
    table = 'season_info'
    print(" Table: ", table)
    combine = "INSERT INTO "+ table + " SELECT * FROM dba." + table + \
              " WHERE dba." + table + ".id NOT IN (SELECT ID FROM " + table + ")"
    print(" " + combine)
    comb_db.execute(combine)

    # Cultivars TABLE
    table = 'cultivars'
    print(" Table: ", table)
    combine = "INSERT INTO "+ table + " SELECT * FROM dba." + table + \
              " WHERE dba." + table + ".id NOT IN (SELECT ID FROM " + table + ")"
    print(" " + combine)
    comb_db.execute(combine)

    # WEATHER TABLE
    table = 'weather'
    print(" Table: ", table)
    combine = "SELECT max(id) FROM " + table
    print("  " + combine)
    c = comb_db.execute(combine)
    max_id = 0
    for row in c:
        max_id = row[0]
    print("   max id: " + str(max_id))
    combine = "SELECT * FROM dba." + table + " WHERE dba." + table + ".timestamp NOT IN " + \
              " (SELECT DISTINCT timestamp FROM " + table + ")"
    print("  " + combine)
    c = comb_db.execute(combine)
    combine = "INSERT INTO " + table + " VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?)"
    print("  " + combine)
    row_added = 0
    weather_id_map = {}
    for row in c:
        max_id += 1
        new_row = list(row)
        new_row[0] = max_id
        comb_db.execute(combine, new_row)
        weather_id_map[row[0]] = max_id
        row_added += 1
    print("  Added %s rows" % str(row_added))
 
    # FILES TABLE
    table = 'files'
    print(" Table: ", table)
    combine = "SELECT max(id) FROM " + table
    print("  " + combine)
    c = comb_db.execute(combine)
    max_id = 0
    for row in c:
        max_id = row[0]
    print("   max id: " + str(max_id))
    combine = "SELECT * FROM dba." + table + " WHERE dba." + table + ".filename NOT IN " + \
              " (SELECT DISTINCT filename FROM " + table + ")"
    print("  " + combine)
    c = comb_db.execute(combine)
    combine = "INSERT INTO " + table + " VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    print("  " + combine)
    row_added = 0
    file_id_map = {}
    for row in c:
        max_id += 1
        new_row = list(row)
        new_row[0] = max_id
        comb_db.execute(combine, new_row)
        file_id_map[row[0]] = max_id
        row_added += 1
    print("  Added %s rows" % str(row_added))
   
    # WEATHER FILE MAP TABLE 
    table = 'weather_file_map'
    print(" Table: ", table)
    combine = "SELECT max(id) FROM " + table
    print("  " + combine)
    c = comb_db.execute(combine)
    max_id = 0
    for row in c:
        max_id = row[0]
    print("   max id: " + str(max_id))
    combine = "SELECT * FROM dba." + table
    print("  " + combine)
    c = comb_db.execute(combine)
    combine = "INSERT INTO " + table + " VALUES(?, ?, ?, ?)"
    print("  " + combine)
    row_added = 0
    for row in c:
        max_id += 1
        new_row = [max_id, file_id_map[row[1]], weather_id_map[row[2]], weather_id_map[row[3]]]
        comb_db.execute(combine, new_row)
        row_added += 1
    print("  Added %s rows" % str(row_added))

    comb_db.commit()
    comb_db.execute("detach database dba")
print("Done")

In [ ]:
import sqlite3
db_name='season_6.db'
comb_db = sqlite3.connect(db_name)
for table in [
    ['season_info','season_info_index'], 
    ['weather', 'weather_index'], 
    ['cultivars', 'cultivars_index'],
    ['files', 'files_index'],
    ['weather_file_map','weather_file_map_index'], 
    ]:
#    c = comb_db.execute('Select * from %s LIMIT 5' % table[0])
#    c = comb_db.execute('Select min(start_time),max(start_time) from %s LIMIT 5' % table[0])
    c = comb_db.execute('Select count(*) from %s' % table[0])
    print(table[0], c.description, len(c.description))
    for row in c:
        print(row)


In [ ]:
import sqlite3
db_name='season_6_2.db'  # Distinct files: 190134
comb_db = sqlite3.connect(db_name)
print("Season Info")
c = comb_db.execute("SELECT count(1),plot_name,season,cultivar_id from season_info group by " + \
                    "plot_name,season,cultivar_id")
cnt = 0
doubles = 0
for row in c:
    cnt += 1
    if row[0] > 1:
        doubles += 1
    if row[0] > 2:
        print('  ERROR: more than double entries')
        print('  ', row)
print("  Number season_info:", cnt, " doubles:", doubles)
print("Weather")
c = comb_db.execute("SELECT count(1),timestamp,temperature,illuminance,precipitation,sun_direction, " + \
                    "wind_speed,wind_direction,relative_humidity from weather group by timestamp")
cnt = 0
doubles = 0
for row in c:
    cnt += 1
    if row[0] > 1:
        doubles += 1
    if row[0] > 2:
        print('  ERROR: more than double entries')
        print('  ', row)
print("  Number weather:", cnt, " doubles:", doubles)
print("Cultivars")
c = comb_db.execute("SELECT count(1),name from cultivars group by name")
cnt = 0
doubles = 0
for row in c:
    cnt += 1
    if row[0] > 1:
        doubles += 1
    if row[0] > 2:
        print('  ERROR: more than double entries')
        print('  ', row)
print("  Number cultivars:", cnt, " doubles:", doubles)
print("Files")
c = comb_db.execute("SELECT count(1),filename,folder from files group by filename,folder")
cnt = 0
doubles = 0
for row in c:
    cnt += 1
    if row[0] > 1:
        doubles += 1
    if row[0] > 2:
        print('  ERROR: more than double entries')
        print('  ', row)
print("  Number files:", cnt, " doubles:", doubles)
c = comb_db.execute("SELECT t.*, f.filename, w1.timestamp min_weather_ts, w2.timestamp max_weather_ts FROM " + \
              "weather_file_map t LEFT JOIN files f ON t.file_id=f.id LEFT JOIN weather w1 ON " +\
              " t.min_weather_id=w1.id LEFT JOIN weather w2 ON t.max_weather_id=w2.id")
print(c.description, len(c.description))
cnt = 0
for row in c:
    cnt += 1
#    print(row)
print("Number to join:", cnt)